In [ ]:
!pip install azure.ai.formrecognizer==3.2.0
!pip install XlsxWriter==3.0.1 


In [ ]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azure.storage.blob import generate_blob_sas, BlobSasPermissions
from datetime import datetime, timedelta

import logging
import json
import os
import logging
from datetime import datetime
from json import JSONEncoder
from azure.core.exceptions import ResourceNotFoundError
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient
import pandas as pd
import io
from azure.ai.formrecognizer import DocumentAnalysisClient
import urllib.parse



In [ ]:
def get_blob_sas(storage_account_name, storage_account_key, storage_container_name, blob_name):
    try:
        #print(account_key)
        sas_blob = generate_blob_sas(account_name= storage_account_name, 
                                    container_name= storage_container_name,
                                    blob_name= blob_name,
                                    account_key= storage_account_key,
                                    permission= BlobSasPermissions(read=True),
                                    start=datetime.utcnow(),
                                    expiry=datetime.utcnow() + timedelta(hours=1))
        return sas_blob
    except Exception as e:
        print(f'{get_blob_sas.__name__}: {str(e)}')

def analyze_document( output_storage_acct, add_keyvalue_pairs, excel_output_folder, endpoint, key, formUrl ):
    try:
        parsed_url = urllib.parse.urlparse(formUrl)
        full_file_name = parsed_url.path.split("/")[-1]
        filename = os.path.splitext(full_file_name)[0]

        document_analysis_client = DocumentAnalysisClient(
            endpoint=endpoint, credential=AzureKeyCredential(key)   
        )
        model="prebuilt-document"
        poller = document_analysis_client.begin_analyze_document_from_url(model,formUrl)
        result = poller.result()

        output_record = generate_excel(result,filename, output_storage_acct, excel_output_folder, add_keyvalue_pairs)
        
        

    except Exception as error:
        output_record =   "Error: " + str(error)
        

    logging.info("Output record: " + output_record)
    return output_record


def generate_excel(result,filename, csvoutputstorage, csvoutputfolder, add_keyvalue_pairs):
    if(add_keyvalue_pairs):
        kvp=get_key_value_pairs(result)
    formtables = {}
    blob_service_client = BlobServiceClient.from_connection_string(csvoutputstorage)
    container_client=blob_service_client.get_container_client(csvoutputfolder)
    output = io.BytesIO()
    writer = pd.ExcelWriter(output, engine='xlsxwriter')
    workbook = writer.book
    merge_format = workbook.add_format({'align': 'center', 'valign': 'vcenter', 'border': 2})

    current_page_num=None
    table_num=1
    for table in result.tables:
        column_row_spans = []
        tableList = [[None for x in range(table.column_count)] for y in range(table.row_count)] 

        for cell in table.cells:
            cellvalue=None
            #only add the cell data if it has some alphanumeric text
            if  (sum( c.isalnum() for c in cell.content) >0):
                #replace selection marks data
                cellvalue=cell.content.replace(":unselected:", "").replace(":selected:", "")
                if(cell.row_span>1 and cell.column_span>1):
                    column_row_spans.append([cell.row_index, cell.column_index, cell.row_index+cell.row_span-1,cell.column_index+cell.column_span-1, cellvalue ])
                elif(cell.row_span>1 and cell.column_span==1):
                    column_row_spans.append([cell.row_index, cell.column_index, cell.row_index+cell.row_span-1,cell.column_index, cellvalue ])
                elif(cell.column_span>1 and cell.row_span==1):
                    column_row_spans.append([cell.row_index, cell.column_index, cell.row_index,cell.column_index+cell.column_span-1, cellvalue ])
                    
            tableList[cell.row_index][cell.column_index] = cellvalue
        
        if current_page_num is None:
            current_page_num = table.bounding_regions[0].page_number
        elif (current_page_num is not None) and (current_page_num == table.bounding_regions[0].page_number):
            table_num +=1
        elif (current_page_num is not None) and (current_page_num != table.bounding_regions[0].page_number):
            table_num =1
            current_page_num = table.bounding_regions[0].page_number


        excel_sheet_name=str(current_page_num) + '_' + str(table_num)
        df = pd.DataFrame.from_records(tableList)
        #set the header row
        df.columns = df.iloc[0] 
        df = df[1:]
        #remove empty rows
        df = df[df.any(axis=1)]        
        #write to excel only if dataframe has some data
        if not(df.empty):
            if(add_keyvalue_pairs and current_page_num in kvp ):
                for key in kvp[current_page_num]:
                    if not(key in df.columns ):
                        df[key]= kvp[current_page_num][key]
            df.to_excel(writer, sheet_name=excel_sheet_name, index=False)
            worksheet = writer.sheets[excel_sheet_name]
            if( len(column_row_spans) >0):  
                for x in column_row_spans:
                    worksheet.merge_range(x[0],x[1],x[2],x[3],x[4], merge_format)


    #excel output
    excelname=filename +'.xlsx'
    writer.close()
    logging.info("writing excel for : " + excelname)
    container_client.upload_blob(name=excelname,data=output.getvalue(),overwrite=True)

    return 'Individual table per sheet has been generated sucecssfully in Excel:' +excelname


def get_key_value_pairs(result):
    kvp = {}
    pagekvp = {}
    pagelen= len(result.pages)
    pagenum=None
    currpagenum=None
    for kv_pair in result.key_value_pairs:
        if pagenum is None:
            pagenum=kv_pair.key.bounding_regions[0].page_number
        elif (pagenum is not None) and (pagenum != kv_pair.key.bounding_regions[0].page_number):
            pagekvp[pagenum]=kvp
            kvp = {}
            pagenum=kv_pair.key.bounding_regions[0].page_number

        if kv_pair.key:
            if kv_pair.value:
                kvp[kv_pair.key.content] = kv_pair.value.content
    pagekvp[pagenum]=kvp
    return pagekvp

Pre-Requisites
1. Create Form Recognizer Service
2. Create Storage account and store the input forms 
3. Create Key Vault Serviceand confirgure with the following secrets
    -  formrecogkey : Form Recogbizer Key
    - formrecogurl  : Form Recogbizer End Point URL
    - inputcontainerurl : URL of the Storage Container that has the forms data
    - saconstr      : Connection String of the Storage account that has the forms data
    - sakey         : Key of the Storage account that has the forms data

Generated Excel spreagsheets are stored in folder specified in 'excel_output_folder'

In [ ]:
KEY_VAULT_NAME = '{Name of the Key Valut}'
STORAGE_ACCOUNT_NAME = '{Name of the Storage Acct}'
STORAGE_ACCOUNT_KEY = 'sakey'
STORAGE_CONN_STRING = 'saconstr'
FORMS_RECOGNIZER_URL_SECRET_NAME = 'formrecogurl'
FORMS_RECOGNIZER_KEY_SECRET_NAME = 'formrecogkey'
STORAGE_CONTAINER_URL ='inputcontainerurl'

container_name = '{Name of the Container that has the input forms}'
excel_output_folder ="output/data"
add_keyvalue_pairs = False

account_key = TokenLibrary.getSecret(f'{KEY_VAULT_NAME}.vault.usgovcloudapi.net', STORAGE_ACCOUNT_KEY, KEY_VAULT_NAME)
print(account_key)

storage_connection_string = TokenLibrary.getSecret(f'{KEY_VAULT_NAME}.vault.usgovcloudapi.net', STORAGE_CONN_STRING , KEY_VAULT_NAME)
print(storage_connection_string)

input_container_url = TokenLibrary.getSecret(f'{KEY_VAULT_NAME}.vault.usgovcloudapi.net', STORAGE_CONTAINER_URL , KEY_VAULT_NAME)
print(input_container_url)



frendpoint = TokenLibrary.getSecret(f'{KEY_VAULT_NAME}.vault.usgovcloudapi.net', FORMS_RECOGNIZER_URL_SECRET_NAME , KEY_VAULT_NAME)
frkey = TokenLibrary.getSecret(f'{KEY_VAULT_NAME}.vault.usgovcloudapi.net', FORMS_RECOGNIZER_KEY_SECRET_NAME , KEY_VAULT_NAME)
print(frendpoint)
print(frkey)



In [ ]:
blob_service_client = BlobServiceClient.from_connection_string(storage_connection_string)
container_client = blob_service_client.get_container_client(container_name)

# List the blobs in the container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print("\t" + blob.name)

    blob_sas = get_blob_sas(storage_account_name=STORAGE_ACCOUNT_NAME, storage_account_key=account_key, storage_container_name=container_name, blob_name=blob.name)
    print("\t blob_sas:" + blob_sas)

    blob_sas_url = f'{input_container_url}/{blob.name}?{blob_sas}'
    print("\t blob_sas_url:" + blob_sas_url)


    output_record = analyze_document( storage_connection_string, add_keyvalue_pairs, excel_output_folder,endpoint=frendpoint, key=frkey,  formUrl=blob_sas_url )  
    
    
    print(output_record)


END